In [28]:
# Imports
%load_ext autoreload
%autoreload 2 
# !apt-get install -y xvfb
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
from tqdm.notebook import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## LOAD FILE W WEIBEL

In [29]:
# Load the e5 csv and add weibel gen (if uncommented)
# orig_df = pd.read_csv(os.path.abspath("/home/sneha/e5lungairwaysvida_20140211.csv"))
orig_df = pd.read_csv(os.path.abspath('/home/sneha/e5lungairwaysvida_20140211_weibel.csv')) # original + weibel gen added

/tmp/ipykernel_25249/2955888588.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  orig_df = pd.read_csv(os.path.abspath('/home/sneha/e5lungairwaysvida_20140211_weibel.csv')) # original + weibel gen added


## Remove scan problems

In [30]:
# REMOVE SCAN PROBLEM
ids_w_scan_problem = orig_df.loc[orig_df.scan_problem != 0].idno.unique()
print(f"There are {len(ids_w_scan_problem)} ids with san problems --> remove them")
orig_df = orig_df.loc[~(orig_df.idno.isin(ids_w_scan_problem))]
print(f"There are {orig_df.idno.nunique()} remaining participants")




There are 92 ids with san problems --> remove them
There are 3102 remaining participants


## Remove nans in startbpid / endbpid

In [31]:
# REMOVE NANS in START + ENDBPID
orig_df.dropna(subset=['startbpid', 'endbpid'], inplace=True)
print(f"There are {orig_df.idno.nunique()} remaining participants after dropping NaNs in start/endbpid")
# print null
orig_df.isnull().sum()

/tmp/ipykernel_25249/4115848727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orig_df.dropna(subset=['startbpid', 'endbpid'], inplace=True)


There are 2911 remaining participants after dropping NaNs in start/endbpid


Unnamed: 0                         0
idno                               0
subject_name                       0
subject_id                         0
session_id                         0
                                  ..
innerperimeter_70                  0
outerperimeter_70                  0
innerequivalentcirclediameter_7    0
scan_problem                       0
weibel_generation                  0
Length: 149, dtype: int64

## Remove unneccesary cols

In [32]:
# drop unneccesary cols

useful_cols = ['idno',
 'anatomicalname',
 'centerlinelength','avginnerarea','lobe',
 'sublobe',
 'endbpid',
 'startbpid',
 'angle',
 'dircosx',
 'dircosy',
 'dircosz', 'weibel_generation']

orig_df = orig_df[useful_cols]
print(len(orig_df))



964751


## QC paths only

In [33]:
# KEEP ONLY QC PATHS
qc_names = [x for x in orig_df.anatomicalname.unique() if ('unnamed' not in x.lower()) & (x != '-')]
orig_df['QC'] = False

print(qc_names)
orig_df.loc[orig_df.anatomicalname.isin(qc_names), 'QC'] = True
display(orig_df)

def _travel_two(df, extend_names=['LB1', 'LB10', 'RB1', 'RB4', 'RB10'], extend_gen=2):
    print('start',df.QC.sum())
    startbpids = df.loc[df.anatomicalname.isin(extend_names)]['endbpid'].to_list()
    print(f"Extending {extend_gen} generations from {extend_names} which have segment ids = {startbpids}")
    qc_children = startbpids
    for i in range(extend_gen):
        new_children = df.loc[df.startbpid.isin(qc_children)]['endbpid'].to_list()
        qc_children+=new_children
#         print('Adding a generation', qc_children)
    print('end', len(qc_children))
    return qc_children
    
          
          
for i, participant_df in tqdm(orig_df.groupby('idno')):
    print(participant_df.idno.unique())
    qc_children = _travel_two(participant_df,extend_names=['LB1', 'LB10', 'RB1', 'RB4', 'RB10'], extend_gen=2)
    orig_df.loc[(orig_df.idno==int(participant_df.idno.unique())) & (orig_df.endbpid.isin(qc_children)), 'QC']= True

display(orig_df)


['Trachea', 'LMB', 'RMB', 'BronInt', 'RUL', 'LLB6', 'LUL', 'RB4+5', 'RB1', 'LB6', 'LB4+5', 'RLL7', 'RB6', 'RB4', 'RB5', 'RB2', 'RB3', 'LLB', 'LB3', 'LB1+2', 'LB5', 'LB4', 'RLL', 'RB7', 'LB8', 'LB2', 'LB1', 'RB10', 'LB10', 'LB9', 'RB8', 'RB9', 'RB2a']


,idno,anatomicalname,centerlinelength,avginnerarea,lobe,sublobe,endbpid,startbpid,angle,dircosx,dircosy,dircosz,weibel_generation,QC
0,3010007,Trachea,104.96200,351.51300,0.0,0.0,1.0,-1.0,NaN,-0.024446,0.229788,0.972934,0,True
1,3010007,LMB,51.01420,148.68100,0.0,0.0,3.0,1.0,140.0360,0.608512,0.045926,0.792214,1,True
2,3010007,RMB,25.44420,215.56900,0.0,0.0,2.0,1.0,134.8050,-0.694225,-0.047878,0.718164,1,True
3,3010007,BronInt,29.83190,111.98300,0.0,0.0,4.0,2.0,160.3120,-0.417837,0.018167,0.908341,2,True
4,3010007,RUL,16.64940,75.69660,3.0,0.0,5.0,2.0,98.5825,-0.813123,-0.038720,-0.580802,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992295,8024995,-,6.14524,6.48287,5.0,16.0,334.0,314.0,165.1320,0.164399,0.000000,0.986394,12,False
992296,8024995,-,15.64100,6.87548,5.0,19.0,335.0,317.0,156.6690,0.074125,0.667124,0.741249,12,False
992297,8024995,-,7.54570,6.98296,5.0,19.0,336.0,317.0,161.6230,0.205076,0.205076,0.957020,12,False
992298,8024995,-,3.32710,5.57702,2.0,9.0,337.0,327.0,143.1220,-0.742781,-0.371391,0.557086,12,False


  0%|          | 0/2911 [00:00<?, ?it/s]

[3010007]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [10.0, 19.0, 54.0, 61.0, 91.0]
end 43
[3010058]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [30.0, 37.0, 45.0, 49.0, 174.0]
end 48
[3010082]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [17.0, 21.0, 50.0, 57.0, 160.0]
end 45
[3010090]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [16.0, 20.0, 75.0, 82.0, 153.0]
end 41
[3010112]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [18.0, 29.0, 39.0, 168.0, 205.0]
end 46
[3010139]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [18.0, 42.0, 52.0, 106.0, 353.0]
end 45
[3010155]
start 31
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have se

[3011518]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [54.0, 62.0, 108.0, 149.0, 293.0]
end 43
[3011569]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [10.0, 25.0, 59.0, 68.0, 268.0]
end 47
[3011585]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [8.0, 43.0, 52.0, 56.0, 222.0]
end 45
[3011593]
start 30
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [18.0, 30.0, 48.0, 87.0, 90.0]
end 46
[3011607]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [21.0, 23.0, 29.0, 59.0, 117.0]
end 47
[3011615]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [19.0, 20.0, 53.0, 107.0, 149.0]
end 45
[3011623]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have se

...

[8017581]
start 30
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [9.0, 35.0, 40.0, 56.0]
end 16
[8017590]
start 31
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [24.0, 27.0, 46.0, 55.0, 120.0]
end 46
[8017611]
start 30
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [11.0, 27.0, 34.0, 63.0, 83.0]
end 43
[8017620]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [30.0, 20.0, 46.0, 63.0, 296.0]
end 45
[8017689]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [33.0, 20.0, 29.0, 40.0, 211.0]
end 45
[8017700]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [9.0, 25.0, 27.0, 67.0, 90.0]
end 45
[8017719]
start 31
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = 

end 49
[8019282]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [10.0, 23.0, 51.0, 85.0, 88.0]
end 33
[8019320]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [15.0, 19.0, 77.0, 90.0, 156.0]
end 47
[8019339]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [9.0, 22.0, 49.0, 115.0, 223.0]
end 46
[8019371]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [91.0, 96.0, 103.0, 106.0, 157.0]
end 45
[8019428]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [20.0, 24.0, 52.0, 107.0, 197.0]
end 41
[8019452]
start 31
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [32.0, 25.0, 29.0, 71.0, 170.0]
end 43
[8019479]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which

[8022186]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [8.0, 21.0, 26.0, 112.0, 261.0]
end 45
[8022194]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [11.0, 35.0, 53.0, 61.0, 139.0]
end 45
[8022224]
start 31
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [30.0, 21.0, 26.0, 113.0, 184.0]
end 43
[8022259]
start 31
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [22.0, 27.0, 33.0, 53.0, 128.0]
end 41
[8022283]
start 31
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [31.0, 35.0, 45.0, 69.0, 248.0]
end 41
[8022330]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [40.0, 47.0, 54.0, 69.0, 173.0]
end 44
[8022364]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have se

[8024383]
start 31
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [8.0, 21.0, 28.0, 199.0, 255.0]
end 43
[8024405]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [17.0, 53.0, 67.0, 154.0, 238.0]
end 43
[8024480]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [21.0, 48.0, 58.0, 59.0, 291.0]
end 45
[8024502]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [39.0, 55.0, 77.0, 99.0, 108.0]
end 27
[8024537]
start 32
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [10.0, 17.0, 27.0, 92.0, 109.0]
end 41
[8024561]
start 30
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segment ids = [30.0, 41.0, 50.0, 56.0, 65.0]
end 5
[8024650]
start 31
Extending 2 generations from ['LB1', 'LB10', 'RB1', 'RB4', 'RB10'] which have segm

,idno,anatomicalname,centerlinelength,avginnerarea,lobe,sublobe,endbpid,startbpid,angle,dircosx,dircosy,dircosz,weibel_generation,QC
0,3010007,Trachea,104.96200,351.51300,0.0,0.0,1.0,-1.0,NaN,-0.024446,0.229788,0.972934,0,True
1,3010007,LMB,51.01420,148.68100,0.0,0.0,3.0,1.0,140.0360,0.608512,0.045926,0.792214,1,True
2,3010007,RMB,25.44420,215.56900,0.0,0.0,2.0,1.0,134.8050,-0.694225,-0.047878,0.718164,1,True
3,3010007,BronInt,29.83190,111.98300,0.0,0.0,4.0,2.0,160.3120,-0.417837,0.018167,0.908341,2,True
4,3010007,RUL,16.64940,75.69660,3.0,0.0,5.0,2.0,98.5825,-0.813123,-0.038720,-0.580802,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992295,8024995,-,6.14524,6.48287,5.0,16.0,334.0,314.0,165.1320,0.164399,0.000000,0.986394,12,False
992296,8024995,-,15.64100,6.87548,5.0,19.0,335.0,317.0,156.6690,0.074125,0.667124,0.741249,12,False
992297,8024995,-,7.54570,6.98296,5.0,19.0,336.0,317.0,161.6230,0.205076,0.205076,0.957020,12,False
992298,8024995,-,3.32710,5.57702,2.0,9.0,337.0,327.0,143.1220,-0.742781,-0.371391,0.557086,12,False


In [34]:
orig_df = orig_df[orig_df.QC]
print("Keeping only QC paths", len(orig_df))

Keeping only QC paths 173610


## Remove all orphans

In [41]:
def _remove_orphans(df):
    orphan_check = np.zeros_like(df.endbpid.values, dtype=bool)
    groups = df.groupby('idno')
    i = 0
    for name, group in tqdm(groups, desc='Iterating ids'):
        orphan_check[i:i+len(group)] =\
        np.isin(group.startbpid.values, group.endbpid.values) | \
        (group.startbpid == -1)
        i = i+len(group)
    return orphan_check


orpha_check = _remove_orphans(orig_df)
orig_df.loc[~orpha_check & (orig_df.idno ==3010007)]

Iterating ids:   0%|          | 0/2911 [00:00<?, ?it/s]

,idno,anatomicalname,centerlinelength,avginnerarea,lobe,sublobe,endbpid,startbpid,angle,dircosx,dircosy,dircosz,weibel_generation,QC
15,3010007,RLL7,9.66940,53.6968,5.0,0.0,16.0,8.0,162.0230,-0.214115,0.160586,0.963518,4,True
16,3010007,RB6,9.27264,28.8561,5.0,15.0,17.0,8.0,98.9529,0.075810,0.985527,-0.151620,4,True
21,3010007,RB2,9.97400,22.7958,3.0,11.0,22.0,11.0,117.7730,-0.610960,0.678844,-0.407307,4,True
22,3010007,RB3,16.80600,25.8426,3.0,12.0,23.0,11.0,136.8160,-0.584858,-0.793736,-0.167102,4,True
23,3010007,LLB,23.56850,43.9090,2.0,0.0,25.0,12.0,150.0120,0.335999,-0.044800,0.940796,4,True
27,3010007,LB3,9.70734,24.7908,1.0,3.0,29.0,14.0,151.2640,0.685189,-0.548151,-0.479632,4,True
28,3010007,LB1+2,7.50000,27.4757,1.0,0.0,28.0,14.0,147.2810,0.000000,0.000000,-1.000000,4,True
89,3010007,LB10,13.01880,16.4163,2.0,9.0,91.0,49.0,156.9830,-0.165948,0.373383,0.912714,6,True
90,3010007,LB9,6.89167,18.9902,2.0,8.0,90.0,49.0,163.2190,0.498273,0.249136,0.830455,6,True
111,3010007,RB8,12.38830,17.3935,5.0,17.0,112.0,60.0,154.6250,-0.832050,-0.443760,0.332820,7,True


In [37]:
def remove_all_orphans(orig_df):
    there_are_orphans = True
    i = 0
    while there_are_orphans:
        i+=1
        
        orphan_check = _remove_orphans(orig_df)
        num_orphan = (orphan_check==0).sum()
        
        if num_orphan ==0:
            print(f"Iter {i}: THERE ARE NO ORPHANS. Final df length {len(orig_df)}")
            there_are_orphans = False
            break
        else:
            print(f"Iter {i}:There are {num_orphan} orphans, REMOVING THEM")
            orig_df = orig_df.loc[orphan_check]
            print(f"Df length {len(orig_df)}")
    return orig_df


df_no_orphans = remove_all_orphans(orig_df)
    
print(len(no_or))

Iterating ids:   0%|          | 0/2911 [00:00<?, ?it/s]

Iter 1:There are 30225 orphans, REMOVING THEM
Df length 143385


Iterating ids:   0%|          | 0/2911 [00:00<?, ?it/s]

Iter 2:There are 26269 orphans, REMOVING THEM
Df length 117116


Iterating ids:   0%|          | 0/2911 [00:00<?, ?it/s]

...

Iterating ids:   0%|          | 0/2908 [00:00<?, ?it/s]

Iter 7:There are 4 orphans, REMOVING THEM
Df length 72638


Iterating ids:   0%|          | 0/2907 [00:00<?, ?it/s]

Iter 8: THERE ARE NO ORPHANS. Final df length 72638


NameError: name 'no_or' is not defined

## Add coords of tree

In [21]:
# ADD COorodinates

def _get_coords(single_test, start_x=0, start_y=0,start_z=0, start_loc=-1, x_sf=1, y_sf=1, z_sf=1):
    '''
    single_test: single participant tree dataframe
    startx/y/z: coords of carina to start
    start_loc: 1 = endbpid corresponding to startx,y,z 1=carina, -1 = top of trachea  
    x/y/z_sf: scale factors based on voxel dimensions mm    
    
    '''
    assert single_test.startbpid.min() == start_loc, 'The dataframe entered has rows before start_loc'
    # gets coords of ENDBPID
    # THIS HAS -1 on the Y!
    # delta x, delta y, delta z for each of the segments --> scaled by pixel scale factor to get to 1:1:1 visualisation
    x_diff= (single_test.centerlinelength*single_test.dircosx/x_sf).values
    # running without minus
    y_diff = (single_test.centerlinelength*single_test.dircosy/y_sf).values
    z_diff = (single_test.centerlinelength*single_test.dircosz/z_sf).values
    endbpid = single_test.endbpid.values
    idno = single_test.idno.values
    startbpid = single_test.startbpid.values
#     print(list(zip(endbpid, x_diff, y_diff, z_diff)))
    
    x = np.zeros_like(x_diff)
    y = np.zeros_like(x_diff)
    z = np.zeros_like(x_diff)
    # for each segment, if we're at the start location, add start location to delta x,y,z
    # otherwise, find the parent x,y,z value (done sequentially) and add on parent location
    for i in range(len(endbpid)):
        if startbpid[i] == start_loc:
#             print("adding start loc", start_x, start_y, start_z)
            x[i] = x_diff[i] + start_x
            y[i] = y_diff[i] + start_y
            z[i] = z_diff[i] + start_z
#             print(i, x[i], y[i], z[i])
        else:
            # find parent endbpid and get the corresponding x,y,z
            parent = np.where(endbpid == startbpid[i])
#             print('parent',parent)
            x[i] = x_diff[i] + x[parent] 
            y[i] = y_diff[i] + y[parent] 
            z[i] = z_diff[i] + z[parent] 
#             print("parent", parent, "i", i, x[i], y[i], z[i])
    # put into dataframe
    coords_df = pd.DataFrame({ "endbpid": endbpid, "x": x, "y": y, "z":z})
    tree_w_coords= single_test.merge(coords_df, how="left", on='endbpid')
    
    
    
    def get_parent_coords(x, col='x'):
        '''
        IMP: the df must be called tree_w_coords
        x: row in df.apply function
        col: str x,y,z to tell which parent coord to get
        It will not fill in start loc - do manually after
        '''
        # x to the df titled **tree_w_coords** get all the endbpids/ segment labels 

        v = np.array(tree_w_coords['endbpid'].to_list()) 
        # np.isin - check T/F for the location of the parent segment for the row x will be an array (F, T,F,F..)
        if np.any(np.isin(v,x.startbpid)):
            # provided we can find the parent, get the index of that parent (the first True in the isin array)
            index = np.argmax(np.isin(v,x.startbpid)) # find row that contains the parent branch (prev bpid)
            try:
                # try to get the parent row from the df
                row_parent = tree_w_coords.iloc[index] 
            except:
                print("Help", index, len(x))
            # return x,y,z loc of parent
            return row_parent[col]
        else:
#             print(f"No parent {col} coord", x.to_list())
            return np.nan
    
    
    tree_w_coords["parent_loc_x"]=tree_w_coords.apply(lambda x: get_parent_coords(x,'x'), axis=1)
    tree_w_coords["parent_loc_y"]=tree_w_coords.apply(lambda x: get_parent_coords(x,'y'), axis=1)
    tree_w_coords["parent_loc_z"]=tree_w_coords.apply(lambda x: get_parent_coords(x,'z'), axis=1)

    # fill parent loc for start posn with start loc (Nan from the df.apply func)
    tree_w_coords.loc[tree_w_coords.startbpid == start_loc, 'parent_loc_x']= start_x
    tree_w_coords.loc[tree_w_coords.startbpid ==  start_loc,'parent_loc_y']= start_y
    tree_w_coords.loc[tree_w_coords.startbpid ==  start_loc, 'parent_loc_z']= start_z
    
    
    
    return tree_w_coords




In [27]:

print(len(df_no_orphans))


72638


In [22]:
groups = df_no_orphans.groupby('idno')

name, group = list(groups)[0]
df_w_coords = _get_coords(group)

for name, group in tqdm(list(groups)[1:]):
    df_w_coords = pd.concat([df_w_coords, _get_coords(group)], axis=0,ignore_index=True)
    print(len(df_w_coords))
    

  0%|          | 0/2906 [00:00<?, ?it/s]

38
61
82
111
126
155
177
209
238
261
292
315
341
362
385
408
431
458
487
516
539
562
597
620
651
686
709
735
758
786
823
852
883
904
935
964
994
1025
1048
1070
1093
1115
1137
1160
1190
1213
1243
1272
1301
1313
1328
1348
1386
1408
1428
1456
1479
1502
1529
1560
1589
1614
1643
1672
1706
1726
1749
1764
1787
1815
1828
1841
1866
1889
1911
1948
1970
2001
2017
2036
2059
2082
2105
2126
2163
2181
2208
2231
2254
2285
2308
2337
2360
2384
2412
2427
2450
2464
2485
2508
2531
2552
2571
2594
2620
2647
2674
2691
2723
2746
2777
2800
2829
2852
2875
2912
2943
2958
2981
3009
3046
3077
3108
3131
3152
3183
3210
3232
3247
3275
3306
3329
3351
3380
3403
3422
3444
3468
3508
3542
3562
3592
3627
3658
3687
3718
3747
3767
3797
3820
3849
3875
3904
3927
3958
3987
4013
4037
4063
4087
4115
4144
4172
4203
4234
4247
4270
4293
4316
4337
4359
4389
4425
4452
4481
4512
4541
4572
4609
4632
4653
4665
4695
4718
4749
4772
4795
4819
4852
4874
4896
4915
4938
4961
4994
5023
5046
5077
5125
5148
5171
5194
5217
5246
5275
5301
5330
5351


35749
35773
35796
35827
35850
35873
35897
35921
35950
35982
36003
36024
36055
36076
36099
36126
36145
36178
36191
36222
36257
36280
36301
36324
36353
36380
36408
36429
36463
36492
36513
36544
36563
36586
36609
36632
36655
36686
36698
36718
36750
36773
36800
36823
36856
36878
36913
36938
36960
36981
37020
37043
37076
37098
37128
37152
37183
37212
37233
37254
37277
37304
37324
37347
37369
37392
37422
37444
37467
37488
37510
37541
37564
37587
37610
37630
37656
37687
37718
37741
37761
37784
37802
37825
37844
37868
37890
37917
37954
37975
37998
38021
38052
38075
38104
38129
38148
38177
38208
38231
38244
38273
38304
38327
38356
38369
38389
38412
38442
38463
38482
38513
38536
38565
38594
38620
38641
38669
38692
38721
38752
38767
38793
38816
38839
38862
38885
38918
38943
38974
39004
39041
39063
39095
39118
39133
39164
39185
39214
39237
39260
39289
39310
39329
39352
39383
39414
39445
39468
39491
39519
39548
39582
39597
39620
39639
39660
39686
39709
39732
39761
39784
39804
39827
39851
39880
3990

70172
70191
70218
70248
70266
70289
70311
70342
70358
70380
70401
70424
70455
70486
70515
70538
70561
70584
70599
70632
70655
70686
70715
70738
70761
70792
70815
70845
70864
70901
70932
70962
70989
71012
71036
71063
71077
71100
71133
71154
71176
71195
71218
71249
71271
71293
71315
71342
71365
71395
71416
71445
71468
71491
71522
71551
71574
71597
71620
71639
71662
71685
71708
71731
71760
71781
71804
71833
71856
71879
71902
71922
71952
71981
72004
72025
72048
72070
72093
72122
72145
72182
72211
72234
72271
72299
72328
72343
72364
72397
72409
72432
72461
72482
72509
72531
72551
72573
72607
72638


## Save file

In [26]:
df_w_coords

,idno,anatomicalname,centerlinelength,avginnerarea,lobe,sublobe,endbpid,startbpid,angle,dircosx,dircosy,dircosz,weibel_generation,QC,x,y,z,parent_loc_x,parent_loc_y,parent_loc_z
0,3010007,Trachea,104.96200,351.51300,0.0,0.0,1.0,-1.0,NaN,-0.024446,0.229788,0.972934,0,True,-2.565859,24.119008,102.121099,0.000000,0.000000,0.000000
1,3010007,LMB,51.01420,148.68100,0.0,0.0,3.0,1.0,140.0360,0.608512,0.045926,0.792214,1,True,28.476894,26.461861,142.535262,-2.565859,24.119008,102.121099
2,3010007,RMB,25.44420,215.56900,0.0,0.0,2.0,1.0,134.8050,-0.694225,-0.047878,0.718164,1,True,-20.229859,22.900801,120.394207,-2.565859,24.119008,102.121099
3,3010007,BronInt,29.83190,111.98300,0.0,0.0,4.0,2.0,160.3120,-0.417837,0.018167,0.908341,2,True,-32.694730,23.442751,147.491745,-20.229859,22.900801,120.394207
4,3010007,RUL,16.64940,75.69660,3.0,0.0,5.0,2.0,98.5825,-0.813123,-0.038720,-0.580802,2,True,-33.767869,22.256133,110.724202,-20.229859,22.900801,120.394207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72633,8024995,-,14.60270,9.30348,4.0,13.0,47.0,22.0,158.2210,-0.663792,-0.414870,0.622305,5,True,-59.383771,12.724356,169.130566,-49.690616,18.782578,160.043233
72634,8024995,-,17.16660,7.13352,4.0,13.0,84.0,46.0,156.1650,-0.811301,0.579501,0.077267,6,True,-72.914768,30.722827,161.369639,-58.987489,20.774765,160.043233
72635,8024995,-,3.05052,5.25465,4.0,13.0,85.0,46.0,141.4660,-0.800000,0.000000,-0.600000,6,True,-61.427905,20.774765,158.212921,-58.987489,20.774765,160.043233
72636,8024995,-,5.10382,6.54452,4.0,13.0,87.0,47.0,165.9940,-0.615457,-0.615457,0.492366,6,True,-62.524953,9.583174,171.643513,-59.383771,12.724356,169.130566


In [13]:
df_w_coords.to_csv('/home/sneha/e5_cleaned_QC.csv',index=False)